In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
import sys
%config InlineBackend.figure_format = 'retina'

In [0]:
transforms = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [40]:
train_data = datasets.CIFAR10("Cifar/",train=True,transform=transforms,download=True)
test_data = datasets.CIFAR10("Cifar/",train=False,transform=transforms,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
train_loader = torch.utils.data.DataLoader(train_data,batch_size=64,num_workers=0,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=20,num_workers=0,shuffle=False)

In [42]:
torch.cuda.is_available()

True

In [0]:
images,labels = next(iter(train_loader))

In [44]:
images.shape

torch.Size([64, 3, 32, 32])

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 32x32x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 10)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 64 * 4 * 4)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

# create a complete CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (dropout): Dropout(p=0.25)
)


In [0]:
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [50]:
n_epochs = 30
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
  model.cuda()

for idx in range(n_epochs):
    train_loss = 0.0
    test_loss = 0.0
    for data, labels in train_loader:
        if train_on_gpu:
            data, labels = data.cuda(), labels.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss= criterion(output,labels)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print("\rEpoch {}/{} \t Training Loss: {:.6f}".format(idx,n_epochs,train_loss))

Epoch 0/30 	 Training Loss: 1.017419
Epoch 1/30 	 Training Loss: 0.993548
Epoch 2/30 	 Training Loss: 0.970909
Epoch 3/30 	 Training Loss: 0.944078
Epoch 4/30 	 Training Loss: 0.926361
Epoch 5/30 	 Training Loss: 0.902615
Epoch 6/30 	 Training Loss: 0.885001
Epoch 7/30 	 Training Loss: 0.866697
Epoch 8/30 	 Training Loss: 0.846309
Epoch 9/30 	 Training Loss: 0.827201
Epoch 10/30 	 Training Loss: 0.813305
Epoch 11/30 	 Training Loss: 0.796012
Epoch 12/30 	 Training Loss: 0.777327
Epoch 13/30 	 Training Loss: 0.762648
Epoch 14/30 	 Training Loss: 0.746774
Epoch 15/30 	 Training Loss: 0.729428
Epoch 16/30 	 Training Loss: 0.718930
Epoch 17/30 	 Training Loss: 0.702704
Epoch 18/30 	 Training Loss: 0.686690
Epoch 19/30 	 Training Loss: 0.678095
Epoch 20/30 	 Training Loss: 0.663241
Epoch 21/30 	 Training Loss: 0.648803
Epoch 22/30 	 Training Loss: 0.638151
Epoch 23/30 	 Training Loss: 0.625551
Epoch 24/30 	 Training Loss: 0.613599
Epoch 25/30 	 Training Loss: 0.601236
Epoch 26/30 	 Training

In [0]:
import numpy as np

In [0]:
batch_size=20
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

In [62]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    
    _, pred = torch.max(output, 1)    
    
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.730472

Test Accuracy of airplane: 84% (843/1000)
Test Accuracy of automobile: 88% (883/1000)
Test Accuracy of  bird: 71% (718/1000)
Test Accuracy of   cat: 55% (556/1000)
Test Accuracy of  deer: 60% (600/1000)
Test Accuracy of   dog: 67% (671/1000)
Test Accuracy of  frog: 77% (770/1000)
Test Accuracy of horse: 84% (844/1000)
Test Accuracy of  ship: 79% (790/1000)
Test Accuracy of truck: 77% (775/1000)

Test Accuracy (Overall): 74% (7450/10000)
